Worksheet
=========

## Objectives

Examine how real-time energy prices (from [ISO New England](https://www.iso-ne.com/isoexpress/) or [PJM](https://dataviewer.pjm.com/dataviewer/pages/public/lmp.jsf)) are established

* Formulate the linear optimization problem in Python
* Discuss several scenarios
* Perhaps try our hand as Market Participants, if time allows

## Optimal Power Flow (OPF) problems

| Category  | Description                      | Per       | Unit      |
| --------- | -------------------------------- | --------- | --------- |
| output    | dispatch instructions            | generator | MW        |
| output    | locational marginal prices (LMP) | bus       | $/MWh     |
| input     | demand/load                      | bus       | MW        |
| input     | supply quantity, price           | offer     | MW, $/MWh |
| input     | reactance, capacity              | line      | Ω, MW     |
| input     | line-bus topology                | -         | -         |


### AC optimal power flow (OPF) problem:

$$
\min_{S^{\text{gen}} \in \mathbb{C}^{M+N}, ~ v \in \mathbb{C}^M} ~ \sum_a c_a(S_a^{\text{gen}})
\\
S_{ab} = v_a (v_a^* - v_b^*) / z_{ab}^* \quad \text{(power flow on line $ab$)}
\\
S_a^{\text{gen}} = \sum_{b} S_{ab} + S_a^{\text{load}} \quad \text{(power balance at bus $a$)}
\\
\vert S_{ab}\vert \leq \bar{S}_{ab} \quad \text{(line capacity)}
\\
\underline{S}_a \leq S_a^{\text{gen}} \leq \bar{S}_a \quad \text{(injection bounds)}
\\
0 < \underline{v}_a \leq |v_a| \leq \bar{v}_a \quad \text{(voltage limits)}
\\
v_0 = 1 + 0i \quad \text{voltage at reference bus}
$$

### Linearized OPF problem:

Simplifications:

| Symbol            | Linearized OPF | AC OPF analog                 | Assumption                              | Unit     |
| ----------------- | -------------- | ----------------------------- | --------------------------------------- | -------- |
| bus injection     | $p_a$          | $\operatorname{real}(S_{a}^{\text{gen}})$  | $\operatorname{imag}(S_{a}^{\text{gen}}) \approx 0$  | MW       |
| line flow         | $f_{ab}$       | $\operatorname{real}(S_{ab})$ | $\operatorname{imag}(S_{ab}) \approx 0$ | MW       |
| bus voltage angle | $\theta_a$     | $\operatorname{angle}(v_a)$   | $\operatorname{abs}(v_a) \approx 1$     | rad      |
| line reactance    | $x_{ab}$       | $\operatorname{imag}(z_{ab})$ | $\operatorname{real}(z_{ab}) \approx 0$ | $\Omega$ |

Resulting formulation:

$$
\min_{p, ~ f, ~ \theta} ~ \sum_a c_a p_a
\\
f_{ab} = (\theta_a - \theta_b) / x_{ab} \quad \text{(power flow on line $ab$)}
\\
p_a = \sum_{b} f_{ab} \quad \text{(power balance at bus $a$)}
\\
\vert f_{ab}\vert \leq \bar{f}_{ab} \quad \text{(line capacity)}
\\
0 \leq p_a \leq \bar{p}_a \quad \text{(injection bounds)}
\\
\theta_0 = 0 \quad \text{angle at reference bus}
$$

Associated marginal price at bus $b$:

$$
\text{LMP}_b~[\$/\text{MWh}] = \frac{\partial~\text{total cost [\$/h]}}{\partial~\text{load}_b~[\text{MW}]}
$$

Run every 5 minutes to clear the Real-Time Energy Market.

## Activity 0: Introduction to Deepnote and Jupyter notebooks

* Create a Deepnote account
* Start your Deepnote computer
* Run cells
* Add new cells

## Activity 1: Meet the input data format [5 minutes]

> Aim to become familiar the format our of data tables (`buses`, `generators`, `lines`, `offers`) and to interact with them.

* Open `activity1-inputs.ipnb` and examine the dataframes defined within.
* Execute each cell (and the entire notebook) to get acquainted with the [Jupyter](https://docs.jupyter.org/en/latest/) notebook interface.
* Identify where each column shows up in the plot.

Next, we'll consider the processing between the tables and the plot.

## Activity 2: Define the linearized power flow constraints [10-15 minutes]

> The linearized OPF involves two sets of equality constraints. One is already implemented, and we'll complete the other.

1. Open `activity2-opf.ipnb`, and scan its contents: You see an incomplete OPF model implemented in [CVXPY](https://www.cvxpy.org/).
2. Study the power balance constraint, provided. You might like to execute the cell (after its predecessors) and examine the results of any subexpressions that are unclear.
3. Try to complete the following elements:
   - The power flow constraints (relating line flows and bus voltage angles).
   - The sum in the `cp.Minimize(...)` objective function
   - Upper bounds on the decision variables (lower bounds are provided)
   - The voltage constraint at the reference bus
4. Run the entire script: If you don't get a total cost of `$17479.90/h`, try to spot your error.
5. On the plot, check that:
   - All constraints are satisfied.
   - You can't spot a cheaper dispatching generators to meet all loads.
   - The sum of flows around each simple loop is zero.

## Activity 3: Locational marginal prices [5 mins]

> In the previous activity, the LMPs appeared as a byproduct of the solver. Let's confirm that these prices meet our definition.

1. Select any bus (e.g., `A`) and make a note of its LMP.
2. Increase the load at your chosen bus by 1MW and re-solve the OPF problem.
3. The difference between the new optimal objective and the original (`$17479.90/h`) should match the LMP you wrote down.
4. You might like to repeat this for another node.

> How might you use this procedure to price/value line capacity? 

## Activity 4: Congestion scenarios [20 mins]

All of these activities make use of `congestion-scenarios.ipynb`.

### Scenario 1: No congestion [5 mins]

* Study the notebook and execut it.
* Verify that there is no line congestion and no price separation.
* Check that the LMPs satisfy our definition at each node.

### Scenario 2: Price separation [5 mins]

* Reduce the capacity of line `Bus1-Bus2` to 30MW and resolve the notebook.
* Verify that prices now separate.
* Again, verify that the LMPs satisfy the definition at each node.

### Scenario 3: Congestion premium [5 mins]


### Scenario 4: Reverse congestion [5 mins]


## Activity 5: Market participation [10 mins]

> Let's try this in groups of 2-3, say.

1. Open `pjm5bus.ipynb` and each pick a different generator.
2. **Confidentially**, write down up to three supply offers (price \& quantity) for your generator (reflecting its current capacity) on a secret slip of paper.
3. Update the offers paper and re-solve the OPF. Record the change in revenue, relative to the baseline configuration.
4. Record your best revenue increase relative to the baseline, over two or three rounds.

## Review questions

### Marginal prices

1. Identify products with near-zero marginal price?
2. What is a (locational) marginal energy price (LMP)?
3. Why might LMPs vary across a network?
4. Why do LMPs vary over time?
5. Which operating costs are not captured by marginal pricing?
6. How are these additional costs covered in practice?

### Optimal power flow

1. What are the decision variables of the OPF problem? How are they used?
2. What are the constraints? How do they associate with network elements?
3. What is the objective function?
4. What are the key assumptions in the linearized OPF model?
5. Are we always able to solve the DC-OPF?
6. Why not solve the AC-OPF instead?
7. How does power flow relate to bus voltage angles?

### Network economics

1. Do the LMP have economic significance at a bus with zero load? With zero generation capacity?
2. How can load payments exceed generation payments if energy losses are neglected?

### Unit commitment

1. Which variable in the commitment problem do not appear in the pricing problem?
2. Which constraints in the commitment problem do not appear in the pricing problem?
3. Why are energy prices not determined from the commitment problem?
4. Which of a generator's operating costs are not captured by the pricing problem?

### Implementations

1. Why are LMPs not universally adopted?

### Extensions

1. What might you change in our OPF model to accommodate battery energy storage?
2. Is the location of the reference bus significant?

## Unit commitment

<!-- Krishnamurthy et al. (2016) -->

| Cost | Description |
|---|---|
| start-up costs | to attain synchronized state |
| no-load costs | to remain synchronized |
| dispatch costs | of energy injection |
| shut-down costs | to eliminate operating costs |

## Extensions for further reading

* Incorporating transmission losses into the linearized OPF model.
* Decomposition LMPs into energy, congestion, and loss components.
* Financial Transmission Rights (FTRs) on congestion rents.
* Uplift payments for transmission losses.
* Load forecasting techniques.
* Systematic detection of market power.

## References

* [Fu & Li (2006) _Different Models and properties on LMP calculations_](https://doi.org/10.1109/PES.2006.1709536)
  - A description of LMP formulations, including the pricing of congestion and losses
* [Li & Bo (2010) _Small Test Systems for Power System Economic Studies_](https://doi.org/10.1109/PES.2010.5589973)
  - A description of the PJM 5-bus model
* [CVXPY documentation](https://www.cvxpy.org/)
* [Deepnote documentation](https://deepnote.com/docs/)
* [Jupyter documentation](https://docs.jupyter.org/en/latest/)
